# An example scraper covering multiple 'next' pages

The code below can be copied and adapted to create your own scraper.

The first part installs all the libraries. I've kept this separate to the other parts so that you don't have to install them every time you want to run the scraper itself.

The second code block uses those functions to scrape a number of pages linked by 'next' buttons. To do this it creates some special custom functions

## Defining your own custom functions 

Often in scraping we will need to run the same block of code over and over again. In this situation it is useful to store that code in a function of its own that we can call when we want to re-use it.

To create a custom function in Python you use the command `def` followed by the name you want to give your function. You then need some parentheses containing the names you want to give its ingredients (the parameters). 

Finally, you need to add a **colon**.

For example: `def dosomething(withthis):`

When you press enter the next line will be indented and any indented lines that follow will be stored within the function.

However, the code inside the function will not run *until the function is called*.

You call the function as you do any other. For example, to call the `dosomething` function used as an example above, you would simply type something like:

`dosomething(putyourspecificthinghere)`

This will take the variable `putyourspecificthinghere` and **pass** it to that function (as its one ingredient). Within the function this is assigned to a new **local variable** (called `withthis` in the example above) which only exists while that function is running.

## Functions that `return` information

Because information within a function only exists while it is running, some functions pass information *back* so that it can be stored.

This is normally done through a `return()` command. Here's an example:

```python
def addup(num1, num2):
  total = num1+num2
  return(total)

gettingaresult = addup(5,10)
```

In the example above a function is defined which **returns** the total of the two numbers it is supplied with ("passed"). 

When the function is run, with the numbers 5 and 10, it is run as part of a line creating a new variable, so the result returned is stored in that variable.

In [1]:
#install the libraries 
#scraperwiki is a library for scraping webpages
!pip install scraperwiki
import scraperwiki
#lxml.html is used to convert it into xml (more structured)
import lxml.html
#cssselect is used to drill down into that and find data in tags
!pip install cssselect
import cssselect
#the pandas library which is used to work with data - we call it 'pd' here so we have to type less!
import pandas as pd

     |████████████████████████████████| 1.1MB 5.9MB/s 
     |████████████████████████████████| 471kB 22.3MB/s 
  Created wheel for scraperwiki: filename=scraperwiki-0.5.1-cp36-none-any.whl size=6547 sha256=01097c94f47613874e66f503690dfa2bddc49dff88357f4c41f712d3e3b3740a
  Stored in directory: /root/.cache/pip/wheels/6a/6e/60/e13b585339206922e816bb90c355b79aa077ab2b15d7cc26a7
  Created wheel for alembic: filename=alembic-1.4.0-py2.py3-none-any.whl size=157561 sha256=67d963c7c15fbbe7cd6284556dfbf2ffca3abe9698dd39cab60c5c2b6f358c52
  Stored in directory: /root/.cache/pip/wheels/26/5e/30/84097f2b9e8215a03af766c647ac6dbef8f2f522af31e8c192
  Created wheel for Mako: filename=Mako-1.1.1-cp36-none-any.whl size=75410 sha256=15302a443f186b5531dedd1ca98dbef68aeec3567efe5fe454ecde9b33af5b80
  Stored in directory: /root/.cache/pip/wheels/ff/50/a9/0ddeed8679a1fb65bf4677cb9c92701828b2c1821e22ef72fd
Successfully built scraperwiki alembic Mako


In [15]:
#This is a full URL for testing
testurl = "https://www.officialcharts.com/charts/singles-chart/20171222/7501/"

#Create a dataframe to store the data we are about to scrape
#It has 5 columns, supplied as a list
#We call this dataframe 'df'
df = pd.DataFrame(columns=["title","artist","label","url","date"])
print(df)

#Function to find the 'next' link
def findnext(url):
  #Scrape the html at that url
  html = scraperwiki.scrape(url)
  # turn our HTML into an lxml object
  root = lxml.html.fromstring(html) 
  #Grab any elements within <a ... class="next">
  nexts = root.cssselect('a.next')
  #There are two, the same, we grab the href="" attribute of the first
  nextlink = nexts[0].attrib['href']
  #Print it - note it's a relative link so it has to be combined with the base URL
  print(nextlink)
  #Return the full link
  return("https://www.officialcharts.com"+nextlink)

#Function to scrape the info from the page itself
def scrapepage(url):
  #This specifies that we want it to treat df as a global variable
  #Otherwise it will treat it as a local variable that only exists within this function
  global df
  #Scrape the html at that url
  html = scraperwiki.scrape(url)
  # turn our HTML into an lxml object
  root = lxml.html.fromstring(html) 
  #Grab the elements we want
  titles = root.cssselect('div.title a')
  artists = root.cssselect('div.artist a')
  labels = root.cssselect('div.label-cat span')
  positions = root.cssselect('span.position')
  listlength = len(titles)
  for i in range(0,listlength):
    #each number in the list generated is called i as it loops
    print(i)
    #we access the item at that position in the list, and grab its text contents
    title = titles[i].text_content()
    artist = artists[i].text_content()
    label = labels[i].text_content()
    position = positions[i].text_content()
    #print them all
    print(title, artist, label, position)
    #Now we need to store it in that variable called 'df' 
    #We also store the url that we've been using throughout
    #Let's extract the date from that too
    date = url.replace("https://www.officialcharts.com/charts/singles-chart/","") #replace the first part of the url with nothing
    date = date.replace("/7501","") #replace the end of the url with nothing
    df = df.append({
      "title" : title,
      "artist" : artist,
      "label" : label,
      "position" : position,
      "url" : url,
      "date" : date
      }, ignore_index=True)

#Run both functions 5 times to get 5 links - change the range to get more pages
for i in range(0,5):
  #Run the first function we defined, with testurl
  #We want the URL to change each time it runs, so we overwrite it each time
  testurl = findnext(testurl)
  print(testurl)
  #Then we run the second function we defined, to scrape that url
  scrapepage(testurl)


#This scraper is a little inefficient as it scrapes each page twice
# - first to get the data and then again to get the next link
# A more efficient approach would do both at the same time but this would make more complex code and I wanted to keep it simpler in this example

Empty DataFrame
Columns: [title, artist, label, url, date]
Index: []
/charts/singles-chart/20180104/7501
https://www.officialcharts.com/charts/singles-chart/20180104/7501
0
PERFECT ED SHEERAN ASYLUM 1
1
LAST CHRISTMAS WHAM RCA 2
2
RIVER EMINEM FT ED SHEERAN INTERSCOPE 3
3
ALL I WANT FOR CHRISTMAS IS YOU MARIAH CAREY COLUMBIA 4
4
FAIRYTALE OF NEW YORK POGUES FT KIRSTY MACCOLL WARNER BROS 5
5
MAN'S NOT HOT BIG SHAQ ISLAND 6
6
DO THEY KNOW IT'S CHRISTMAS BAND AID MERCURY 7
7
ANYWHERE RITA ORA ATLANTIC 8
8
ROCKIN' AROUND THE CHRISTMAS TREE BRENDA LEE MCA 9
9
MERRY CHRISTMAS EVERYONE SHAKIN' STEVENS RCA 10
10
STEP INTO CHRISTMAS ELTON JOHN MERCURY 11
11
HAVANA CAMILA CABELLO FT YOUNG THUG EPIC/SYCO MUSIC 12
12
IT'S BEGINNING TO LOOK A LOT LIKE MICHAEL BUBLE REPRISE 13
13
DRIVING HOME FOR CHRISTMAS CHRIS REA WARNER BROS 14
14
I WISH IT COULD BE CHRISTMAS EVERYDAY WIZZARD EMI 15
15
MERRY XMAS EVERYBODY SLADE UMTV 16
16
IT'S THE MOST WONDERFUL TIME OF THE YEAR ANDY WILLIAMS SONY MUSIC 17
17
I 

In [11]:
#Once the loop has finished we can take a look at the data
print(df)

                               title  ... position
0                            PERFECT  ...        1
1                     LAST CHRISTMAS  ...        2
2                              RIVER  ...        3
3    ALL I WANT FOR CHRISTMAS IS YOU  ...        4
4              FAIRYTALE OF NEW YORK  ...        5
..                               ...  ...      ...
195                        QUESTIONS  ...       96
196           CAN'T STOP THE FEELING  ...       97
197                          FRIENDS  ...       98
198                        SEPTEMBER  ...       99
199                          THUNDER  ...      100

[200 rows x 6 columns]


In [0]:
#And we can export it
df.to_csv("scrapeddata.csv")